In [ ]:
!rm -rf MP_DATA 

In [ ]:
!unzip /content/drive/MyDrive/MP_DATA.zip

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
  inflating: MP_DATA/y/44/28.npy     
  inflating: MP_DATA/y/44/29.npy     
  inflating: MP_DATA/y/44/3.npy      
  inflating: MP_DATA/y/44/4.npy      
  inflating: MP_DATA/y/44/5.npy      
  inflating: MP_DATA/y/44/6.npy      
  inflating: MP_DATA/y/44/7.npy      
  inflating: MP_DATA/y/44/8.npy      
  inflating: MP_DATA/y/44/9.npy      
   creating: MP_DATA/y/45/
  inflating: MP_DATA/y/45/0.npy      
  inflating: MP_DATA/y/45/1.npy      
  inflating: MP_DATA/y/45/10.npy     
  inflating: MP_DATA/y/45/11.npy     
  inflating: MP_DATA/y/45/12.npy     
  inflating: MP_DATA/y/45/13.npy     
  inflating: MP_DATA/y/45/14.npy     
  inflating: MP_DATA/y/45/15.npy     
  inflating: MP_DATA/y/45/16.npy     
  inflating: MP_DATA/y/45/17.npy     
  inflating: MP_DATA/y/45/18.npy     
  inflating: MP_DATA/y/45/19.npy     
  inflating: MP_DATA/y/45/2.npy      
  inflating: MP_DATA/y/45/20.npy     
  inflating: MP_DATA/y/45/21.npy  

In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-7ff46dff-b126-bafc-bb5d-1d7b3f19e66c)


In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

     |████████████████████████████████| 394.3 MB 12 kB/s 
     |████████████████████████████████| 394.3 MB 17 kB/s 
     |████████████████████████████████| 32.7 MB 218 kB/s 
     |████████████████████████████████| 462 kB 48.2 MB/s 
     |████████████████████████████████| 2.9 MB 19.2 MB/s 
     |████████████████████████████████| 3.8 MB 38.2 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68714 sha256=ce3aa577b6baef8f4ee7023dcc9704d86f78589785865fbe790fe5e877690e42
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0


In [ ]:
import cv2
import numpy as np
import os
import datetime
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
DATA_PATH = os.path.join('MP_DATA')
actions = np.array(open('/content/drive/MyDrive/classes.txt', 'r').read().split('\n'))
number_of_sequences = 100
every_sequence_length = 30
sequences, labels = [], []

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'r': 16,
 's': 17,
 't': 18,
 'u': 19,
 'w': 20,
 'y': 21,
 'z': 22}

In [ ]:
for action in actions:
    for sequence in range(number_of_sequences):
        window = []
        for frame_num in range(every_sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
sequences[0]

[array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.39542124,  0.63193005,
         0.        ,  0.48866868,  0.62292951, -0.0

In [ ]:
X = np.array(sequences)

In [ ]:
X

array([[[ 0.        ,  0.        ,  0.        , ...,  0.35959291,
          0.48585761, -0.0107872 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.34653437,
          0.48195836, -0.00795674],
        [ 0.        ,  0.        ,  0.        , ...,  0.37665778,
          0.484368  , -0.01713235],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.37632918,
          0.50157231,  0.0304199 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.37413582,
          0.49998099,  0.02536973],
        [ 0.        ,  0.        ,  0.        , ...,  0.37121937,
          0.49954426,  0.02845585]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.28854802,
          0.64280248,  0.04286661],
        [ 0.        ,  0.        ,  0.        , ...,  0.28911313,
          0.63620001,  0.04674198],
        [ 0.        ,  0.        ,  0.        , ...,  0.29142988,
          0.59800792,  0.03665687],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [ ]:
X.shape

(2300, 30, 126)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
y.shape

(2300, 23)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
#_____________________________________________________________________________
# X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
# X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
#_____________________________________________________________________________
X_train_and_val, X_test, y_train_and_val, y_test = train_test_split(X,y, train_size=0.9)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_and_val,y_train_and_val, test_size=0.2)

In [ ]:
X_train.shape

(1656, 30, 126)

In [ ]:
X_valid.shape

(414, 30, 126)

In [ ]:
X_test.shape

(230, 30, 126)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
batch_size = 128    
epochs = 1000

log_dir = os.path.join('/content/drive/MyDrive/Logs', datetime.datetime.now().strftime("%Y.%m.%d-%H.%M") + '--batch__' + str(batch_size) + '--epochs__' + str(epochs))
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

# ______________________________________
# model = Sequential()
# model.add(LSTM(64, return_sequences=False, activation='relu', input_shape=(30,126)))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
!nvidia-smi

Sun Dec 19 23:25:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    79W / 149W |    141MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model.fit(X_train, y_train, epochs=epochs, shuffle=True, batch_size=batch_size, validation_data= (X_valid, y_valid), callbacks=[tb_callback])

Epoch 1/1000
13/13 [==============================] - 9s 384ms/step - loss: 3.1336 - categorical_accuracy: 0.0478 - val_loss: 3.1135 - val_categorical_accuracy: 0.0894
Epoch 2/1000
13/13 [==============================] - 3s 207ms/step - loss: 3.0827 - categorical_accuracy: 0.0748 - val_loss: 2.9805 - val_categorical_accuracy: 0.1184
Epoch 3/1000
13/13 [==============================] - 3s 211ms/step - loss: 3.0198 - categorical_accuracy: 0.1082 - val_loss: 3.0292 - val_categorical_accuracy: 0.1111
Epoch 4/1000
13/13 [==============================] - 3s 202ms/step - loss: 3.0187 - categorical_accuracy: 0.1149 - val_loss: 2.8869 - val_categorical_accuracy: 0.1135
Epoch 5/1000
13/13 [==============================] - 3s 209ms/step - loss: 2.8382 - categorical_accuracy: 0.1245 - val_loss: 2.7874 - val_categorical_accuracy: 0.1498
Epoch 6/1000
13/13 [==============================] - 3s 207ms/step - loss: 2.7072 - categorical_accuracy: 0.1535 - val_loss: 2.5224 - val_categorical_accuracy:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            48896     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 23)                759       
Total params: 204,119
Trainable params: 204,119
Non-trainable params: 0
__________________________________________________

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

print("Generate predictions for 30 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions.shape)

Evaluate on test data
230/230 [==============================] - 5s 20ms/step - loss: 0.2923 - categorical_accuracy: 0.9522
test loss, test acc: [0.2923068404197693, 0.9521738886833191]
Generate predictions for 30 samples
predictions shape: (230, 23)


In [ ]:
model.save(os.path.join('/content/drive/MyDrive/Logs/Weights', datetime.datetime.now().strftime("%Y.%m.%d-%H.%M") + '--batch__' + str(batch_size) + '--epochs__' + str(epochs), 'action.h5'))